In [ ]:
# Importing important libraries
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation, GlobalAveragePooling2D, add
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam, SGD
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.models import load_model
from keras.callbacks import *
from keras.regularizers import l2
from keras import backend as k

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

import imgaug as ia
from imgaug import augmenters as iaa

# import wget

import glob
import pathlib
import cv2
import zipfile
from sklearn.metrics import confusion_matrix, classification_report
import itertools
import six

num_classes = 200
num_train = 90000
# input image dimensions
img_height, img_width = 64,64
# The images are RGB.
img_channels = 3
num_validation = 10000
batch_size = 60

In [ ]:
""" Mount gdrive on colab"""
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
"""
      upload the zip file in gdrive. Mount it on colab and unzip it using the 
      following command
"""
!unzip './drive/My Drive/image-detect.zip' -d './drive/My Drive/'

In [ ]:
"""
      Architeture for VGG16
"""

_input_ = Input(shape=(img_height, img_width, img_channels)) 

conv1  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu" , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(_input_)
conv2  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu" , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(conv1)
pool1  = MaxPooling2D((2, 2))(conv2)

conv3  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu" , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(pool1)
conv4  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu" , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(conv3)
pool2  = MaxPooling2D((2, 2))(conv4)

conv5  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu" , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(pool2)
conv6  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu" , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(conv5)
conv7  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(conv6)
pool3  = MaxPooling2D((2, 2))(conv7)

conv8  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu" , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(pool3)
conv9  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu" , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(conv8)
conv10 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu" , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(conv9)
pool4  = MaxPooling2D((2, 2))(conv10)

conv11 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu" , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(pool4)
conv12 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu" , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(conv11)
conv13 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu" , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(conv12)
pool5  = MaxPooling2D((2, 2))(conv13)

flat   = Flatten()(pool5)
dense1 = Dense(4096, activation="relu")(flat)
dense2 = Dense(4096, activation="relu")(dense1)
output = Dense(200, activation="softmax")(dense2)

vgg16_model  = Model(inputs=_input_, outputs=output)
model = vgg16_model

model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
"""Architecture for ResNet-34
"""

# # Model building
ip = Input(shape=(img_height, img_width, img_channels))

# Block 1
layer0 = Conv2D(32, (3,3), padding='same', kernel_initializer="VarianceScaling"
                    ,kernel_regularizer=tf.keras.regularizers.l2(2e-4))(ip)
layer0 = BatchNormalization()(layer0)
layer0 = Activation('relu')(layer0)

skip_connection_1 = layer0

# # Block 2

layer1 = Conv2D(128, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer0)
layer1 = BatchNormalization()(layer1)
layer1 = Activation('relu')(layer1)

layer2 = Conv2D(128, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer1)
layer2 = BatchNormalization()(layer2)
layer2 = Activation('relu')(layer2)

layer3 = Conv2D(128, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer2)
layer3 = BatchNormalization()(layer3)
layer3 = Activation('relu')(layer3)

layer4 = Conv2D(128, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer3)
layer4 = BatchNormalization()(layer4)
layer4 = Activation('relu')(layer4)

layer5 = concatenate([skip_connection_1, layer4])
layer5 = BatchNormalization()(layer5)
layer5 = Activation('relu')(layer5)
layer5 = MaxPooling2D(pool_size=(2, 2))(layer5)

skip_connection_2 = layer5

# # Block 3

layer6 = Conv2D(256, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer5)
layer6 = BatchNormalization()(layer6)
layer6 = Activation('relu')(layer6)

layer7 = Conv2D(256, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer6)
layer7 = BatchNormalization()(layer7)
layer7 = Activation('relu')(layer7)

layer8 = Conv2D(256, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer7)
layer8 = BatchNormalization()(layer8)
layer8 = Activation('relu')(layer8)

layer9 = Conv2D(256, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer8)
layer9 = BatchNormalization()(layer9)
layer9 = Activation('relu')(layer9)

layer10 = concatenate([skip_connection_2, layer9])
layer10 = BatchNormalization()(layer10)
layer10 = Activation('relu')(layer10)
layer10 = MaxPooling2D(pool_size=(2, 2))(layer10)

skip_connection_3 = layer10


# # Block 4

layer11 = Conv2D(512, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer10)
layer11 = BatchNormalization()(layer11)
layer11 = Activation('relu')(layer11)

layer12 = Conv2D(512, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer11)
layer12 = BatchNormalization()(layer12)
layer12 = Activation('relu')(layer12)

layer13 = Conv2D(512, (3,3), padding='same',kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer12)
layer13 = BatchNormalization()(layer13)
layer13 = Activation('relu')(layer13)

layer14 = Conv2D(512, (3,3), padding='same',kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer13)
layer14 = BatchNormalization()(layer14)
layer14 = Activation('relu')(layer14)

layer15 = concatenate([skip_connection_3, layer14])
layer15 = BatchNormalization()(layer15)
layer15 = Activation('relu')(layer15)
layer15 = MaxPooling2D(pool_size=(2, 2))(layer15)


# #Layer 16
layer16 = Conv2D(num_classes, (1,1), padding='same',kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer15)
layer16 = GlobalAveragePooling2D()(layer16)

# # #Output Layer
output = Activation('softmax')(layer16)


In [ ]:
""" Architecture for DenseNet-50
"""

def space_to_depth_x2(x):
    return tf.nn.space_to_depth(x, block_size=2)

ip = Input(shape=(None, None, img_channels))

# Block 1

layer1 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1' , kernel_initializer="VarianceScaling" , kernel_regularizer=tf.keras.regularizers.l2(2e-4))(ip)
layer1 = BatchNormalization(name='norm_1')(layer1)
layer1 = Activation("relu")(layer1)

layer2 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2' , kernel_initializer="VarianceScaling" , kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer1)
layer2 = BatchNormalization(name='norm_2')(layer2)
layer2 = Activation("relu")(layer2)

layer3 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3' , kernel_initializer="VarianceScaling" , kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer2)
layer3 = BatchNormalization(name='norm_3')(layer3)
layer3 = Activation("relu")(layer3)

layer4 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_4' , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer3)
layer4 = BatchNormalization(name='norm_4')(layer4)
layer4 = Activation("relu")(layer4)

layer5 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_5' , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer4)
layer5 = BatchNormalization(name='norm_5')(layer5)
layer5 = Activation("relu")(layer5)

layer6 = MaxPooling2D(pool_size=(2, 2))(layer5)

skip_connection_1 = layer6

# Block 2

layer7 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_7' , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer6)
layer7 = BatchNormalization(name='norm_7')(layer7)
layer7 = Activation("relu")(layer7)

layer8 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_8' , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer7)
layer8 = BatchNormalization(name='norm_8')(layer8)
layer8 = Activation("relu")(layer8)

layer9 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_9' , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer8)
layer9 = BatchNormalization(name='norm_9')(layer9)
layer9 = Activation("relu")(layer9)

layer10 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_10' , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer9)
layer10 = BatchNormalization(name='norm_10')(layer10)
layer10 = Activation("relu")(layer10)

layer11 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_11' , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer10)
layer11 = BatchNormalization(name='norm_11')(layer11)
layer11 = Activation("relu")(layer11)

layer12 = MaxPooling2D(pool_size=(2, 2))(layer11)

skip_connection_1 = Lambda(space_to_depth_x2)(skip_connection_1)

layer13 = concatenate([skip_connection_1, layer12])

skip_connection_2 = layer13

# Block 3

layer14 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_14' , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer13)
layer14 = BatchNormalization(name='norm_14')(layer14)
layer14 = Activation("relu")(layer14)

layer15 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_15' , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer14)
layer15 = BatchNormalization(name='norm_15')(layer15)
layer15 = Activation("relu")(layer15)

layer16 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_16' , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer15)
layer16 = BatchNormalization(name='norm_16')(layer16)
layer16 = Activation("relu")(layer16)

layer17 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_17' , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer16)
layer17 = BatchNormalization(name='norm_17')(layer17)
layer17 = Activation("relu")(layer17)

layer18 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_18' , kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.l2(2e-4))(layer17)
layer18 = BatchNormalization(name='norm_18')(layer18)
layer18 = Activation("relu")(layer18)

layer19 = MaxPooling2D(pool_size=(2, 2))(layer18)

skip_connection_2 = Lambda(space_to_depth_x2)(skip_connection_2)

layer20 = concatenate([skip_connection_2, layer19])

layer21 = Conv2D(num_classes, (1,1), name='conv_21')(layer20)
layer21 = BatchNormalization(name='norm_21')(layer21)

layer22 = GlobalAveragePooling2D(data_format=None)(layer21)

layer23 = Activation('softmax')(layer22)

output = layer23


In [ ]:
"""
    A simple convnet architecture
"""
"""Block 1"""
model.add(Conv2D(128, (3, 3), strides=(1,1), padding='same', input_shape=(img_height,img_width,img_channels)))
print(model.layers[-1].output_shape)
model.add(BatchNormalization())
print(model.layers[-1].output_shape)
model.add(Conv2D(128, (3, 3), strides=(1,1), padding='same'))
print(model.layers[-1].output_shape)
model.add(BatchNormalization())
print(model.layers[-1].output_shape)
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))
print(model.layers[-1].output_shape)
model.add(Activation('relu'))
print(model.layers[-1].output_shape)

"""Block 2"""
model.add(Conv2D(128, (3, 3), strides=(1,1), padding='same'))
print(model.layers[-1].output_shape)
model.add(BatchNormalization())
print(model.layers[-1].output_shape)
model.add(Conv2D(128, (3, 3), strides=(1,1), padding='same'))
print(model.layers[-1].output_shape)
model.add(BatchNormalization())
print(model.layers[-1].output_shape)
model.add(Activation('relu'))
print(model.layers[-1].output_shape)
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))
print(model.layers[-1].output_shape)

"""Block 3"""
model.add(Conv2D(128, (3, 3), strides=(1,1), padding='same'))
print(model.layers[-1].output_shape)
model.add(BatchNormalization())
print(model.layers[-1].output_shape)
model.add(Conv2D(128, (3, 3), strides=(1,1), padding='same'))
print(model.layers[-1].output_shape)
model.add(BatchNormalization())
print(model.layers[-1].output_shape)
model.add(Activation('relu'))
print(model.layers[-1].output_shape)
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))
print(model.layers[-1].output_shape)

"""Block 4"""
model.add(Conv2D(256, (3, 3), strides=(1,1), padding='same'))
print(model.layers[-1].output_shape)
model.add(BatchNormalization())
print(model.layers[-1].output_shape)
model.add(Conv2D(512, (3, 3), strides=(1,1), padding='same'))
print(model.layers[-1].output_shape)
model.add(BatchNormalization())
print(model.layers[-1].output_shape)
model.add(Activation('relu'))
print(model.layers[-1].output_shape)
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))
print(model.layers[-1].output_shape)

"""Block 5"""
model.add(Flatten())
print(model.layers[-1].output_shape)
model.add(Dense(4096))
print(model.layers[-1].output_shape)
model.add(BatchNormalization())
print(model.layers[-1].output_shape)
model.add(Activation('relu'))
print(model.layers[-1].output_shape)

"""Block Test"""
model.add(Dense(1024))
print(model.layers[-1].output_shape)
model.add(BatchNormalization())
print(model.layers[-1].output_shape)
model.add(Activation('relu'))
print(model.layers[-1].output_shape)

"""Output Layer"""
model.add(Dense(num_classes))
print(model.layers[-1].output_shape)

"""Loss Layer"""
model.add(Activation('softmax'))
print(model.layers[-1].output_shape)

Reading the contents of the file val_annotations.txt where entries are 
separated by '\t'. There are no column names but annotated using 


**names**=['File', 'Class', 'X', 'Y', 'H', 'W'] where File is the file name,

**Class** is the true class label and 'X', 'Y', 'H', 'W' is for the four
numeric values

In [ ]:
val_data = pd.read_csv('./drive/My Drive/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True) 
val_data.head()

In [ ]:
"""
      For data augmentation 
"""
def CustomImageDataGen(input_img):
  """
  Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
  e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second
  image.
  """
  sometimes = lambda aug: iaa.Sometimes(0.5, aug)
  
  seq = iaa.Sequential([
      iaa.Fliplr(0.5), # horizontal flips
      iaa.Flipud(0.2), # vertical flips
      
      # Small gaussian blur with random sigma between 0 and 0.5.
      # But we only blur about 50% of all images.
      sometimes(iaa.GaussianBlur(sigma=(0, 2.0))),
      
      # crop images by -10% to 20% of their height/width
      sometimes(iaa.CropAndPad(
          percent=(-0.1, 0.2),
          pad_mode=ia.ALL,
          pad_cval=(0, 255)
        )),
      
      # Apply affine transformations to some of the images
      # - scale to 80-120% of image height/width (each axis independently)
      # - translate by -20 to +20 relative to height/width (per axis)
      # - rotate by -45 to +45 degrees
      # - shear by -16 to +16 degrees
      # - order: use nearest neighbour or bilinear interpolation (fast)
      # - mode: use any available mode to fill newly created pixels
      #         see API or scikit-image for which modes are available
      # - cval: if the mode is constant, then use a random brightness
      #         for the newly created pixels (e.g. sometimes black,
      #         sometimes white)
      sometimes(iaa.Affine(
          scale={"x": (0.8, 1.5), "y": (0.8, 1.5)},
          translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
          rotate=(-45, 45),
          shear=(-16, 16),
          order=[0, 1],
          cval=(0, 255),
          mode=ia.ALL
      )),
      
      #drop 2-5% percent of the original size, leading to large dropped
      # rectangles.
      sometimes(iaa.CoarseDropout(
                        (0.03, 0.15), size_percent=(0.02, 0.05),
                        per_channel=0.2
                    )),
                
      # Make some images brighter and some darker.
      # In 20% of all cases, we sample the multiplier once per channel,
      # which can end up changing the color of the images.
      sometimes(iaa.Multiply((0.8, 1.2), per_channel=0.2)),
      
      #Improve or worsen the contrast of images.
      #Comment it out after third model run (extreme saturation)
      sometimes(iaa.ContrastNormalization((0.75, 1.5), per_channel=0.5)), 
     ],
     # do all of the above augmentations in random order
     random_order = True) # apply augmenters in random order
  
  output_img = seq.augment_image(input_img)
  return output_img

"""
      rescale is a value by which we will multiply the data before any other 
      processing. Our original images consist in RGB coefficients in the 0-255, 
      but such values would be too high for our models to process 
      (given a typical learning rate), so we target values between 0 and 1 
      instead by scaling with a 1/255. factor.
"""
train_datagen = ImageDataGenerator(rescale=1/255., preprocessing_function = CustomImageDataGen)
valid_datagen = ImageDataGenerator(rescale=1/255.)

**Directory**: Directory where the data is located. If labels is "inferred",
it should contain subdirectories, each containing images for a class. 


**Target_size**: Size to resize images to after they are read from disk.


**Batch_size**: Size of the batches of data taken into account for an epoch.


**Class_mode**: One of "categorical", "binary", "sparse", "input", or None. 


**Default**: "categorical". Determines the type of label arrays that are 
returned: - "categorical" will be one-hot encoded labels 

**shuffle**: Whether to shuffle the data. Default: True. If set to False, sorts the data in alphanumeric order.

**seed**: Optional random seed for shuffling and transformations.

In [ ]:
train_generator = train_datagen.flow_from_directory( r'./drive/My Drive/train/', 
                                                    target_size=(img_width, img_height), 
                                                    batch_size=batch_size, 
                                                    class_mode='categorical', 
                                                    shuffle=True, seed=101)

In [ ]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./drive/My Drive/val/images/', 
                                                         x_col='File', y_col='Class', 
                                                         target_size=(img_width, img_height),
                                                         class_mode='categorical', 
                                                         batch_size=batch_size, 
                                                         shuffle=False, seed=101)

In [ ]:
"""
      For visualizing the augmented images 
"""
x_batch, y_batch = next(train_generator)

fig = plt.figure(figsize=(10, 10))

for i in range(8):
    sub = fig.add_subplot(2, 4, i + 1)
    sub.imshow(x_batch[i,:,:], interpolation='bicubic')

In [ ]:
model = Model(inputs=[ip], outputs=[output])
model.summary()

tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
"""for architecture 1 and VGG_16
"""
# Compile the Model
model.compile(loss='categorical_crossentropy',
              # optimizer= RMSprop(lr= 0.0001, epsilon=1e-08),
              # optimizer= Adam(lr= 0.0001, epsilon=1e-08),
              optimizer = SGD( learning_rate=0.001, momentum=0.9, nesterov=True),
              metrics=['acc'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=10, verbose=1, min_lr=0.001) #on plateaus

earlystop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=20,
                          verbose=1, mode='auto')

checkpoint = ModelCheckpoint(filepath="./drive/My Drive/Network1_colab.hdf5", verbose=1, save_best_only = False, monitor="val_loss")
callbacks_list = [earlystop,reduce_lr,checkpoint]

model = load_model("./drive/My Drive/Network1_colab.hdf5")
model.fit_generator(train_generator,
                    epochs = 75,
                    steps_per_epoch= num_train // batch_size,
                    validation_steps= num_validation // batch_size,
                    validation_data=validation_generator,
                    verbose=1, callbacks=callbacks_list
                   )

In [ ]:
"""for Architecture 2 and architecture 3
"""
reduce_lr = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10,
                          verbose=1, mode='auto')

checkpoint = ModelCheckpoint(filepath="./drive/My Drive/Network2_colab.hdf5", verbose=1, save_best_only=False, monitor="val_loss")

model = load_model("./drive/My Drive/Network2_colab.hdf5" , custom_objects={'tf': tf})
model.compile(loss='categorical_crossentropy',
              # optimizer= Adam(lr= 0.001, epsilon=1e-08),
              optimizer = SGD(learning_rate=0.001, momentum=0.9, nesterov=True),
              metrics=['accuracy'])
callbacks_list = [earlystop,reduce_lr,checkpoint]

model.fit_generator(train_generator, epochs=75, validation_data=validation_generator, verbose=1, callbacks=callbacks_list)

In [ ]:
model = load_model("./drive/My Drive/Network2_colab.hdf5" , custom_objects={'tf': tf})

In [ ]:
model.fit_generator(train_generator, epochs=75, validation_data=validation_generator, verbose=1, callbacks=callbacks_list) # steps_per_epoch=3000, validation_steps=3000,

In [ ]:
"""Evaluating the trained model on validation data"""
validation_steps_per_epoch = np.math.ceil(validation_generator.samples / validation_generator.batch_size)
score = model.evaluate_generator(validation_generator, verbose=1, steps=validation_steps_per_epoch)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

In [ ]:
#Prediction based on validation data
pred=model.predict_generator(validation_generator, steps= np.ceil(num_validation/batch_size), verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)
# Predicted class indices of 1st 10 val images
predicted_class_indices[:10]

In [ ]:
# True class indices of 1st 10 validation images
validation_generator.classes[:10]

In [ ]:
# Predicted classes from their indices
labels = (validation_generator.class_indices)
labels = dict((v,x) for x,v in labels.items())
predictions = [labels[x] for x in predicted_class_indices]
predictions[:10] #first 10 of them


In [ ]:
# Validation class names from words.txt
class_to_name = dict()
file = open('./drive/My Drive/words.txt','r')
data= file.readlines()
for line in data:
  words = line.strip('\n').split('\t')
  class_to_name[words[0]] = words[1].split(',')[0]
file.close()

In [ ]:
# Asserting Validation Class names from words.txt
validation_class_names={}
for _class in validation_generator.class_indices.keys():
  validation_class_names.update({_class : class_to_name[_class]})
  
# Classification Report of val classes
print(classification_report(validation_generator.classes, predicted_class_indices,
                            #target_names=validation_generator.class_indices.keys(),
                            target_names=validation_class_names.values(),
                            digits=4))

In [ ]:
""" A list of all test images"""
classes=[]
data_dir = pathlib.Path('./drive/My Drive/train/')
classes = sorted([item.name for item in data_dir.glob('*')])

In [ ]:
z=[]
file1 = open(r"./drive/My Drive/prediction_network2.txt","w+")
for file in glob.glob("./drive/My Drive/test/images/*.JPEG"):
    images = [cv2.imread(file)]
    processed_image=np.divide(images[:],255.0)
    prediction_probs = model.predict(processed_image)
    prediction = int(np.argmax(prediction_probs.reshape(-1)))
    file1.write('{},{}\n'.format(file[29:], classes[prediction]))
    z.append(classes[prediction])
file1.close()